# Práctica 2: Aprendizaje y selección de modelos de clasificación

#### Minería de Datos: Curso académico 2020-2021

#### Profesorado:
* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

#### Integrantes:
* Gonzalo Pinto Perez
* Yeremi Martin Huaman Torres

# 1.-Preliminares

Indicamos la semilla y las librerias que utilizaremos

In [ ]:
# Third party
from sklearn.base import clone
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import Normalizer
from imblearn.pipeline import make_pipeline
from sklearn.impute import *
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import *
# Local application
import miner_a_de_datos_aprendizaje_modelos_utilidad as utils

In [ ]:
random_state = 27912

# 2. Carga de datos

Se realiza la carga de datos de los conjuntos de datos de Pima Indian Diabetes y Breast Cancer Wisconsin

In [ ]:
filepathDiabetes = "../input/pima-indians-diabetes-database/diabetes.csv"

filepathWisconsin = "../input/breast-cancer-wisconsin-data/data.csv"

indexDiabetes = None
targetDiabetes = "Outcome"

indexWisconsin = "id"
targetWisconsin = "diagnosis"

train_size = 0.7

## 2.1 Pima Indian Diabetes 

In [ ]:
dataDiabetes = utils.load_data(filepathDiabetes, indexDiabetes, targetDiabetes)

Muestreo y separación de datos de Pima Indian Diabetes

In [ ]:
dataDiabetes.sample(5, random_state=random_state)

In [ ]:
(X_Diabetes, y_Diabetes) = utils.divide_dataset(dataDiabetes, targetDiabetes)

In [ ]:
X_Diabetes.sample(5, random_state=random_state)

In [ ]:
y_Diabetes.sample(5, random_state=random_state)

In [ ]:
(X_train_Diabetes, X_test_Diabetes, y_train_Diabetes, y_test_Diabetes) = train_test_split(X_Diabetes, y_Diabetes,
                                                      stratify=y_Diabetes,
                                                      train_size=train_size,
                                                      random_state=random_state)

In [ ]:
X_train_Diabetes.sample(5, random_state=random_state)

In [ ]:
y_train_Diabetes.sample(5, random_state=random_state)

In [ ]:
X_test_Diabetes.sample(5, random_state=random_state)

In [ ]:
y_test_Diabetes.sample(5, random_state=random_state)

## 2.2 Breast Cancer Wisconsin

In [ ]:
dataWisconsin = utils.load_data(filepathWisconsin,indexWisconsin,targetWisconsin)

In [ ]:
dataWisconsin.sample(5, random_state=random_state)

Como se explicado en la practica 1, se elimina el atributo ruidoso 'Unnamed:32' porque se ha introducido erroneamente en la primera linea del csv una coma razon que da a una nueva variable.

In [ ]:
dataWisconsin.drop(['Unnamed: 32'], axis=1, inplace=True)

Nuestra variable objetivo necesita ser 0 o 1 para poder utilizar recall por lo tanto realizamos el siguiente cambio:

In [ ]:
dataWisconsin['diagnosis']=dataWisconsin['diagnosis'].map({'M':1,'B':0})

In [ ]:
dataWisconsin.sample(5, random_state=random_state)

In [ ]:
(X_Wisconsin, y_Wisconsin) = utils.divide_dataset(dataWisconsin, targetWisconsin)

In [ ]:
X_Wisconsin.sample(5, random_state=random_state)

In [ ]:
y_Wisconsin.sample(5, random_state=random_state)

In [ ]:
(X_Wisconsin_train, X_Wisconsin_test, y_Wisconsin_train, y_Wisconsin_test) = train_test_split(X_Wisconsin, y_Wisconsin,
                                                      stratify=y_Wisconsin,
                                                      random_state=random_state,
                                                      train_size=train_size)

In [ ]:
X_Wisconsin_train.sample(5, random_state=random_state)

In [ ]:
X_Wisconsin_test.sample(5, random_state=random_state)

In [ ]:
y_Wisconsin_train.sample(5, random_state=random_state)

In [ ]:
y_Wisconsin_test.sample(5, random_state=random_state)

# 3. Modelos de clasificación supervisada

En este apartado generamos los distintos modelos de clasificación supervisada que utilizaremos para esta práctica, siendo estos:
* el modelo de K números de vecinos 
* el modelo de árbol de decisión
* el modelo adaboost
* el modelo baggin
* el modelo random forest
* el modelo gradient boosting
* el modelo de gradient boosting basado en el histograma respectivamente.

In [ ]:


k_neighbors_model = KNeighborsClassifier()

decision_tree_model = DecisionTreeClassifier(random_state=random_state)

adaboost_model = AdaBoostClassifier(random_state=random_state)

bagging_model = BaggingClassifier(random_state=random_state)

random_forest_model = RandomForestClassifier(random_state=random_state)

gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

hist_gradient_boosting_model = HistGradientBoostingClassifier(random_state=random_state)

# 4. Evaluación de modelos (validación cruzada)

Para evaluar dichos modelos vamos a utilizar validación cruzada para evaluar los modelos de acuerdo con cada una de las configuraciones de hiperparámetros para decidir cuál es la mejor. Para ello utilizaremos GridSearchCV nos mostrara la mejores hiperparametros para cada configuracion de cada algoritmo, ordenados en estos caso por el mejor recall que se puede obtener.




In [ ]:
n_splits = 10
n_repeats = 5

cv = RepeatedStratifiedKFold(n_splits=n_splits,
                             n_repeats=n_repeats,
                             random_state=random_state)

#  5. Selección de modelos

## 5.1 Breast Cancer Wisconsin 

### Pipeline anterior practica - preprocesamiento

En la practica 1 hemos definido este preprocesamiento, y nos ha dado buenos resultados por lo tanto utilizaremos este mismo, junto a la aplicacion de los modelos de clasificacion supervisada anteriormente mencionados 

In [ ]:
preproceserWinsconsin = make_column_transformer(('drop',['radius_mean','radius_se','radius_worst','concave points_mean',
          'concave points_se','concave points_worst']),
                                                remainder='passthrough')

### 5.1.1 K Vecinos mas cercanos

Definimos nuestro estimador

In [ ]:
estimator = clone(k_neighbors_model)

### Normalizar
Es necesario normalizar el conjunto de atributos numericos para el algoritmo de KNN para evitar que los atributos con un mayor rango de valores tengan un mayor peso en el calculo de las distancias y, por tanto, en el algoritmo.

In [ ]:
transformer = Normalizer()

### Discretizar

Se aplicara la discretizacion utilizando las tres estrategias (uniform,quantile,kmeans) y segun el numero de particiones 2,3,4. Se va utilizar discretizacion solo para los algoritmos de KNN y arboles de decision ya que son los algoritmos que mejores resultados se obtiene.

In [ ]:
discretizer = KBinsDiscretizer()

### Pipeline

Establecemos nuestro pipeline aplicando nuestro preprocesamiento, el normalizador, el discretizador con quantile ya que nos dio buenos resultados (estos resultados se explicaran despues de la optimizacion de hiperparametros) y finalmente el estimador

In [ ]:
pipeline_Winsconsin=make_pipeline(preproceserWinsconsin,transformer,discretizer,estimator)

Los dos hiperparametros mas importantes para KNN son
- **weights**: es simplemente la funcion de pesado de los vecinos mas cercanos. Por un lado uniform(todos los puntos se ponderan por igual) y por otro distance (los vecinos más cercanos de un punto tendrán una mayor influencia que los vecinos más alejados) 
- **n_neighbors** : para que no halla la posibilidad de empate ya que nuestra BBDD son pares, el numero de vecinos que elegiremos sera impar. Segun investigaciones en el caso de un pequeño número de vecinos, el ruido tendrá una mayor influencia en el resultado, por el contrario un gran número de vecinos lo hace computacionalmente costoso. Tambien una pequeña cantidad de vecinos tienen un ajuste más flexible, lo que tendrá un sesgo bajo pero una varianza alta, y un gran número de vecinos tendrá un límite de decisión más suave, lo que significa una varianza menor pero un sesgo más alto. Esto se traduce si utilizamos vecinos muy pequeños tendriamos más a underfitting y si utilizamos muy grandes podria llegar al overfitting. Por ello utilizaremos valores no muy pequeños por lo menos 2 digitos entre 11 y 29.

In [ ]:
weights = ["uniform", "distance"]
n_neighbors = [11, 13, 15,17, 21,23,25,27,29]

k_neighbors_clf_Wisconsin = utils.optimize_params(pipeline_Winsconsin,
                                        X_Wisconsin_train, y_Wisconsin_train, cv,
                                        kneighborsclassifier__weights=weights,
                                        kneighborsclassifier__n_neighbors=n_neighbors,
                                        kbinsdiscretizer__n_bins= [2,3,4],
                                        kbinsdiscretizer__strategy= ["uniform", "quantile", "kmeans"],
                                        scoring=['recall'],refit='recall')

Los resultados mas optimos que obtenemos estan alrededor del 11 como el numero de vecinos mas cercanos y como funcion de peso tanto uniform como distance empatando en la mejor configuracion. A lo largo de la lista de resultados que nos devuelve el Grid, podemos observar que a partir de este punto no mejora aumentando el numero de vecinos sino que empeora, por el lado del peso no podemos encontrar una relacion con su orden ya que uniform aparece como la mejor opcion y tambien la peor. 

Por otro lado la discretizacion que obtenemos al aplicar a KNN, respecto a n_bins parece dar mejores resultado un numero medio en lugar uno muy alto o baja, por otro lado parece coincidir utilizando quantile como estrategia para discretizar y la peor uniform.

### 5.1.2 Arboles de decision

In [ ]:
estimator = clone(decision_tree_model)

### Discretizar

Como hemos dicho tambien utilizaremos para los Arboles decisiones ya que son los mas comunes dando un mejor resultado.

In [ ]:
discretizer = KBinsDiscretizer()

### Pipeline

In [ ]:
pipeline_Winsconsin=make_pipeline(preproceserWinsconsin,discretizer,estimator)

Hiperparametro para los arboles de decision:
- **criterion**: Criterio utilizado para medir la calidad de una partición, por un lado gini (la impureza de Gini o el índice de Gini) mide las divergencias entre la distribuciones de probabilidades de los valores de la variable clase y divide un nodo de tal forma que de la menor cantidad de impureza  y entropy (la ganancia de información) divide un nodo de modo que proporcione la mayor cantidad de ganancia de informacion. En todo caso muchos investigadores señalan que en la mayoria de los casos la eleccion de los criterios no marca mucho la diferencia.
- **max_depth** : la profundidad máxima teórica que puede alcanzar un árbol de decisión es uno menos que el número de muestras de entrenamiento, pero ningún algoritmo permitirá llegar a este punto por razones obvias, una de las cuales es el overfitting. Cuanto más profundo permita que nuestro árbol crezca, más complejo se volverá nuestro modelo porque tendrá más divisiones y capturará más información sobre los datos y esta es una de las causas principales del sobreajuste. Por otro lado  tener una profundidad muy baja provocara que nuestro modelo no se ajuste y no podra encontrar el mejor valor. 
- **ccp_alpha**: elegira el subárbol con mayor complejidad de costo que sea más pequeño.

In [ ]:
criterion = ["gini", "entropy"]
max_depth = [ 2, 4, 6, 8,10,12]
ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]

decision_tree_clf_Wisconsin = utils.optimize_params(pipeline_Winsconsin,
                                          X_Wisconsin_train, y_Wisconsin_train, cv,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha,
                                                    kbinsdiscretizer__n_bins= [2,3,4],
                                        kbinsdiscretizer__strategy= ["uniform", "quantile", "kmeans"],
                                                  scoring=['recall'],refit='recall')

En esta caso el criterio mejor valorada es gini, pero podemos observar que los siguientes casos son de entropy y tambien gini esta entre los peores. Por otro lado la profundidad que obtenemos es 2 pero esta empata con un gran numero de otras configuraciones entre un rango par de 2 y 12, por tanto no podemos obtener un conocimiento con esto ya que coinciden todos como la mejor configuracion.
Por ultimo el ccp_alpha que nos indica la tasa se podara, el rango 0.2, 0.3, 0.4 estan entre los primeros y los siguientes aumentando este rango a 0.4 son peores.

Respecto a la discretizacion parece dar mejores resultado con quantile y 2 en el numero de bins y peores con un mayor numero.

### 5.1.3 AdaBoost 

In [ ]:
estimator = clone(adaboost_model)

### Pipeline


In [ ]:
pipeline_Winsconsin=make_pipeline(preproceserWinsconsin,estimator)


Hiperparametro para AdaBoost:
- **base_estimator**: este parametro indica el tipo de estimador que pueden ser arbol de decision, regresor logistico, SVC, etc. No puede ser KNN ya que no se puede asignar el peso a este modelo. En este caso utilizaremos el arbol de decision. 
- **Learning_rate**: parametro que se proporciona para controlar la contribucion de cada clasificador, estos valores comprenderan entre mayor que cero y menor igual a uno. 
- **n_estimatores**: el numero de estimadores que queremos usar en nuestro conjunto de datos, este hiperparametro es el mas importante, la cantidad de arboles agregados al modelo debe ser alta para que el modelo funcione bien.

Para ello estableceremos el valor a partir de 50 hasta 70. En cuanto al resto de metricas utilizaremos las mismo que en el arbol de decision disminuyendo el numero de parametros a probar ya que aumentaria mucho el coste computacional, y lo que buscamos es aprender el uso del grid con adaboost, y la influencia que tiene los hiperparametros para conseguir el mas optimo.

In [ ]:
base_estimator = DecisionTreeClassifier(random_state=random_state)

base_estimator = [base_estimator]
n_estimators= [50,60,70]
learning_rate = [0.0001, 0.1,0.3, 1.0]
criterion = ["gini", "entropy"]
max_depth = [ 2, 4]
ccp_alpha = [0.0, 0.1, 0.2]

adaboost_clf_Wisconsin = utils.optimize_params(pipeline_Winsconsin,
                                     X_Wisconsin_train, y_Wisconsin_train, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__n_estimators=n_estimators,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha,scoring=['recall'],refit='recall')

Primero que todo como se comento se utiliza el algoritmo de arbol de decision, empecemos hablando de estos hiperparametros y como afectan, se observa que ccp_alpha obtiene mejores resultados con 0, que con 0.2. En el caso de la profundidad no aporta mucha informacion ya que solo elegimos dos y los resultados no varian eligiendo uno u otro. Por ultimo el criterio parece haber un mayor numero de entropy que de gini.

Por el lado de los hiperparametro parece haber una relacion entre un mayor numero de estimadores con la mejor configuracion, y en el caso de la tasa de aprendizaje parece ser que una mayor aporta mejores resultados que una menor.

En conclusion los hiperparametros que nos aportan mas informacion a este conjunto de datos parece ser numero de estimadores y la tasa de aprendizaje, y ccp_alpha del arbol de decision.

### 5.1.4 Bagging (Bootstrap Aggregation)

In [ ]:
estimator = clone(bagging_model)

### Pipeline

In [ ]:
pipeline_Winsconsin=make_pipeline(preproceserWinsconsin,estimator)

Hiperparametro para Bagging:
- **n_estimators**: segun reportes cientificos normalmente es buen que se aumenten de 10 o 100. El número de árboles aumenta hasta que se estabiliza el rendimiento del modelo. Podría sugerir que más árboles conducirán a un sobreajuste, aunque este no es el caso. 
- **max_samples**: número de muestras utilizadas para adaptarse a cada árbol de decisiónm normalmente para un conjunto de datos pequeños puede aumentar la variacion de los arboles decision resultantes y podria resultar en un mejor rendimiento. 
- **bootstrap** determina que el muestreo de instancias es con reemplazo. 
- **bootstrap_features** determina que el muestreo de características es con reemplazo.

Bagging puede recibir diferentes nombres segun algunas modificaciones como: Pasting Ensemble (restrigiendo el numero de muestras), Random Subspaces Ensemble(similar a random forest solo que la muestra de arranque es aleatoria y el subconjunto de caracteristicas se selecciona para todo el arbol de decision en lugar de en cada punto de division del arbol), Random Patches Ensemble (este caso implica un ajuste tanto por caracteristicas como por muestras)


In [ ]:
base_estimator = clone(decision_tree_model)
base_estimator = [base_estimator]
max_samples= [0.1,1.1,0.1]
n_estimators=[10,100]
max_features=[1,5,10]
bootstrap= [True,False]
bootstrap_features=[True,False]
criterion = ["gini", "entropy"]

bagging_clf_Wisconsin = utils.optimize_params(pipeline_Winsconsin,
                                    X_Wisconsin_train, y_Wisconsin_train, cv,
                                              baggingclassifier__n_estimators=n_estimators,
                                              baggingclassifier__max_samples=max_samples,
                                              baggingclassifier__max_features=max_features,
                                              baggingclassifier__bootstrap=bootstrap,
                                              baggingclassifier__bootstrap_features=bootstrap_features,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__base_estimator__criterion=criterion,scoring=['recall'],refit='recall')

En este caso los hiperparametros del arbol de decision paracen coincidir que el criterion es entropy para las dos configuraciones primeras.
Por otros lado el resto de los hiperparametros da mejores resultado cuando no hay reemplazo en el muestreo de instancias pero si en el de caracteristicas.

Los mejores resultado nos lo dan las cuatro mejores configuraciones maximo numero de caracteristicas es de 5 pero en este caso parece que no tiene mucha relacion ya que tambien es una de las peores, con maximo de numero de muestras de 0.1 es decir el menor de todas las opciones y el mayor numero de estimadores.

### 5.1.5 Random forests

In [ ]:
estimator = clone(random_forest_model)

### Pipeline

In [ ]:
pipeline_Winsconsin=make_pipeline(preproceserWinsconsin,estimator)

Los hiperparametros de Random Forests: 
- **n_estimators** elegiendo una gran cantidad de estimadores en un modelo de bosque aleatorio no es la mejor idea. Aunque no degradará el modelo, puede ahorrarle la complejidad computacional. 
- **max_features** es recomendable establecer la raíz cuadrada del número de características presentes en el conjunto de datos. El número ideal de max_features generalmente tiende a estar cerca de este valor.

In [ ]:

n_estimators=[50,60,70,80,90]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clf_Wisconsin = utils.optimize_params(pipeline_Winsconsin,
                                          X_Wisconsin_train, y_Wisconsin_train, cv,
                                          randomforestclassifier__criterion=criterion,
                                        randomforestclassifier__n_estimators=n_estimators,
                                          randomforestclassifier__max_features=max_features,scoring=['recall'],refit='recall')

En este caso comprobamos los resultados ya que los mejores resultados dan con los n_estimator menores pero tambien los encontramos entre los peores por lo tanto no podemos establecer una conclusion correcta y tambien utilizando la raiz cuadrada o log2. Por otro lado parace que obtenemos mejores resultados con entropy para los 4 primeras configuraciones.

### 5.1.6 Gradient boosting

In [ ]:
estimator = clone(gradient_boosting_model)

### Pipeline

In [ ]:
pipeline_Winsconsin=make_pipeline(preproceserWinsconsin,estimator)

Los hiperparametros de Gradient boosting: 
- **n_estimators** es bastante robusto en un mayor número de árboles, aún puede sobreajustarse en un punto. 
- **learning_rate** determina el impacto de cada arbol en el resultado final.Generalmente se prefieren valores más bajos ya que hacen que el modelo sea robusto a las características específicas del árbol y, por lo tanto, permiten que se generalice bien. Los valores más bajos requerirían un mayor número de árboles para modelar todas las relaciones y serían computacionalmente costosos. 
- **max_depth** se usa para controlar el overfitting ya que una mayor profundidad permitirá que el modelo aprenda relaciones muy específicas para una muestra en particular.

In [ ]:
n_estimators= [50,60,70]
learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

gradient_boosting_clf_Wisconsin = utils.optimize_params(pipeline_Winsconsin,
                                              X_Wisconsin_train, y_Wisconsin_train, cv,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha,scoring=['recall'],refit='recall')

Observamos que los dos mejores resultados coinciden cuando la profundidad es la mayor es decir 3, tambien cuando ccp_alpha es nulo. Y la tasa de acierto es la mayor. En el caso del criterio utiliza parace que no podemos llegar a una conclusion ya que la aunque mse se encuentre entre las mejores tambien estan entre las peores

### 5.1.7 Histogram gradient boosting

In [ ]:
estimator = clone(hist_gradient_boosting_model)

### Pipeline

In [ ]:
pipeline_Winsconsin=make_pipeline(preproceserWinsconsin,estimator)

Este estimador es mucho mas rapido que Gradiante Boosting pero se utiliza normalmente para grandes conjuntos de datos mayores a 10000.
Los hiperparametros de Gradient boosting: 
- **max_iter** número máximo de iteraciones del algoritmo. 
- **learning_rate** vuelve a ser el mismo que en Gradient Boosting sin optimizar 
- **min_samples_leaf** se recomienda utilizar para conjuntos de datos pequeños.

In [ ]:

max_iter= [10,50,100]
learning_rate = [0.01, 0.05, 0.1]
min_samples_leaf=[10,15,20]


hist_gradient_boosting_clf_Wisconsin = utils.optimize_params(pipeline_Winsconsin,
                                                   X_Wisconsin_train, y_Wisconsin_train, cv,
                                                          histgradientboostingclassifier__max_iter= max_iter,
                                                   histgradientboostingclassifier__learning_rate=learning_rate,
                                                   histgradientboostingclassifier__min_samples_leaf=min_samples_leaf,scoring=['recall'],refit='recall')

Histogram gradient boosting da mejores resultado que el Gradiang Boosting, coinciden en sus resultado con learning_rate una tasa mayor da mejores resultados para este conjunto de datos, una iteracion media de 50 tiene mejores resultado que una alta 100 seguidos de una muy pequeña 10.

## 5.2 Pima Indian Diabetes

Ya no comentaremos sobre los hiperparametros ya que en Wisconsin se comento sobre como ajustarlos y estos los utilizaremos en los otras database, asi que nos centraremos en los resultados y que hiperparametros dan el mas optimo.

### Pipeline anterior practica - preprocesamiento

In [ ]:
#Variables para sustituir ceros
features1 = 'Glucose|BloodPressure'
features2 = 'BMI'
#Variables que no hay que tocar
features3 = 'DiabetesPedigreeFunction|Age'

#Estimador para integuers
replace0Integer_Estimator = make_pipeline(SimpleImputer(strategy="median",missing_values=0 ))
#Estimador para reales
replace0Float_Estimator = make_pipeline(SimpleImputer(strategy="mean",missing_values=0 ))

preproceserDiabetes = make_column_transformer(
    (replace0Integer_Estimator, make_column_selector(pattern= features1)),
    (replace0Float_Estimator, make_column_selector(pattern= features2)),
     ('passthrough', make_column_selector(pattern= features3)))

### 5.2.1 K Vecinos mas cercanos

In [ ]:
estimator = clone(k_neighbors_model)

### Normalizar

In [ ]:
transformer = Normalizer()

### Discretizar

Como se mencionado discretizar parace funcionar mejor con KNN y Arboles de decision.

In [ ]:
discretizer = KBinsDiscretizer()

### Pipeline

In [ ]:
pipeline_Diabetes=make_pipeline(preproceserDiabetes,transformer,discretizer,estimator)

In [ ]:

weights = ["uniform", "distance"]
n_neighbors = [9,11, 13, 15,17, 21,23,25,27,29]

k_neighbors_clf_Diabetes = utils.optimize_params(pipeline_Diabetes,
                                        X_train_Diabetes, y_train_Diabetes, cv,
                                        kneighborsclassifier__weights=weights,
                                                 kbinsdiscretizer__n_bins= [2,3,4],
                                        kbinsdiscretizer__strategy= ["uniform", "quantile", "kmeans"],
                                        kneighborsclassifier__n_neighbors=n_neighbors,scoring=['recall'],refit='recall')

Podemos observar que en esta conjunto de datos tiene una mayor relacion weight, los  mejores resultados son uniform, con vecinos de 11 y 15 respectivamente, pero este orden para en los siguientes que son distance como mejor hiperparametro, podemos decir que solo en casos especificos uniform da mejores resultados que distance. En cuanto al numero de vecinos no podemos hallar una relacion ya que aunque 11 es el mejor resultado podemos observar que 9 esta en el ultimo.

Podemos observa que los mejores resultados son a una mayor cantidad de vecinos, ademas de que el mejor resultado utiliza uniform a diferencia de Wisconsin.

### 5.2.2 Arboles de decision

In [ ]:
estimator = clone(decision_tree_model)

### Discretizar

Como se mencionado discretizar parace funcionar mejor con KNN y Arboles de decision.

In [ ]:
discretizer = KBinsDiscretizer()

### Pipeline

In [ ]:
pipeline_Diabetes=make_pipeline(preproceserDiabetes,discretizer,estimator)

In [ ]:
criterion = ["gini", "entropy"]
max_depth = [ 2, 4, 6, 8,10,12]
ccp_alpha = [0.0, 0.1, 0.2, 0.3, 0.4]

decision_tree_clf_Diabetes = utils.optimize_params(pipeline_Diabetes,
                                          X_train_Diabetes, y_train_Diabetes, cv,
                                          decisiontreeclassifier__criterion=criterion,
                                          decisiontreeclassifier__max_depth=max_depth,
                                          decisiontreeclassifier__ccp_alpha=ccp_alpha,
                                                   kbinsdiscretizer__n_bins= [2,3,4],
                                        kbinsdiscretizer__strategy= ["uniform", "quantile", "kmeans"],
                                                  scoring=['recall'],refit='recall')

En este caso podemos decir que nuestros hiperparametros de ccp_alpha de 0.1 dan los mejores resultados y utilizando la entropia como criterio. Por el lado de maxima profundidad no podemos realizar una mejor descripcion ya que empatan los 5 profundidades que hemos introducido.

Por otro lado la discretizacion da mejores resultado con kmeans y numero intermedio de bins  3, de las cinco mejores configuraciones que podemos observar.

### 5.2.3 Adaboost

In [ ]:
estimator = clone(adaboost_model)

### Pipeline

In [ ]:
pipeline_Diabetes=make_pipeline(preproceserDiabetes,estimator)

In [ ]:


base_estimator = DecisionTreeClassifier(random_state=random_state)

base_estimator = [base_estimator]
n_estimators= [50,60,70]
learning_rate = [0.0001, 0.1,0.3, 1.0]
criterion = ["gini", "entropy"]
max_depth = [ 2, 4]
ccp_alpha = [0.0, 0.1, 0.2]

adaboost_clf_Diabetes = utils.optimize_params(pipeline_Diabetes,
                                     X_train_Diabetes, y_train_Diabetes, cv,
                                     adaboostclassifier__base_estimator=base_estimator,
                                     adaboostclassifier__learning_rate=learning_rate,
                                     adaboostclassifier__n_estimators=n_estimators,
                                     adaboostclassifier__base_estimator__criterion=criterion,
                                     adaboostclassifier__base_estimator__max_depth=max_depth,
                                     adaboostclassifier__base_estimator__ccp_alpha=ccp_alpha,scoring=['recall'],refit='recall')

Observaremos los datos que nos aporta GridSearchCV de la misma manera que con el otro conjunto de datos, en este caso ccp_alpha a 0 da los mejores resultados en todas las configuraciones, en criterion no podemos establecer una relacion clara pero parece ser el mejor gini en los dos casos de mejor configuracion, pero tambien esta entre los peores, la profundidad parece que coinciden en que la 4 en todas las mejores configuraciones.

Por el lado de los hiperparametros de Adaboost, el numero de estimadores no aporta mucha información pero la tasa de aprendizaje si coincide en la menor numero mejor configuracion.

En conclusion la mejores configuraciones son cuando el arbol de decision tiene ccp_alpha a 0 , profundidad maxima de 4, y por el lado de adaboost la tasa de aprendizaje a 0.0001


### 5.2.4 Bagging

In [ ]:
estimator = clone(bagging_model)

In [ ]:
pipeline_Diabetes=make_pipeline(preproceserDiabetes,estimator)

In [ ]:
base_estimator = clone(decision_tree_model)
base_estimator = [base_estimator]
max_samples= [0.1,1.1,0.1]
n_estimators=[10,100]
max_features=[1,5,10]
bootstrap= [True,False]
bootstrap_features=[True,False]
criterion = ["gini", "entropy"]

bagging_clf_Diabetes = utils.optimize_params(pipeline_Diabetes,
                                    X_train_Diabetes, y_train_Diabetes, cv,
                                              baggingclassifier__n_estimators=n_estimators,
                                              baggingclassifier__max_samples=max_samples,
                                              baggingclassifier__max_features=max_features,
                                              baggingclassifier__bootstrap=bootstrap,
                                              baggingclassifier__bootstrap_features=bootstrap_features,
                                    baggingclassifier__base_estimator=base_estimator,
                                    baggingclassifier__base_estimator__criterion=criterion,scoring=['recall'],refit='recall')

En este caso los hiperparametros de arbol de decision nos muestras que las configuraciones con mayor recall son criterion igual a entropy aunque tambien vemos casos en que la aparece en el peor configuracion, en cuanto al muestreo parece ser que en ambos casos dan mejores resultados si no lo hay, por ultimo parece que las mejores soluciones son limitando el numero de caracteristicas a 5 el numero de muestras a 0.1 y utilizando el mayor numero de estimadores.

### 5.2.5 Random Forests

In [ ]:
estimator = clone(random_forest_model)

In [ ]:
pipeline_Diabetes=make_pipeline(preproceserDiabetes,estimator)

In [ ]:
n_estimators=[50,60,70,80,90]
criterion = ["gini", "entropy"]
max_features = ["sqrt", "log2"]

random_forest_clf_Diabetes = utils.optimize_params(pipeline_Diabetes,
                                          X_train_Diabetes, y_train_Diabetes, cv,
                                          randomforestclassifier__criterion=criterion,
                                        randomforestclassifier__n_estimators=n_estimators,
                                          randomforestclassifier__max_features=max_features,scoring=['recall'],refit='recall')

Coincide con Wisconsin en que la entropy nos da mejores resultados, pero tambien se muestra que los resultados con menor numero de estimadores tienen mas frecuencia en las mejores configuraciones.

### 5.2.6 Gradient Boosting

In [ ]:
estimator = clone(gradient_boosting_model)

In [ ]:
pipeline_Diabetes=make_pipeline(preproceserDiabetes,estimator)

In [ ]:
n_estimators= [50,60,70]
learning_rate = [0.01, 0.05, 0.1]
criterion = ["friedman_mse", "mse"]
max_depth = [1, 2, 3]
ccp_alpha = [0.0, 0.1]

gradient_boosting_clf_Diabetes = utils.optimize_params(pipeline_Diabetes,
                                              X_train_Diabetes, y_train_Diabetes, cv,
                                              gradientboostingclassifier__learning_rate=learning_rate,
                                              gradientboostingclassifier__criterion=criterion,
                                              gradientboostingclassifier__max_depth=max_depth,
                                              gradientboostingclassifier__ccp_alpha=ccp_alpha,scoring=['recall'],refit='recall')

Estos casos parecen coincidir con el conjunto de datos de Wisconsin, en lo relativo a la mayor profundidad y una tasa de poda inexistente. En cuanto al criterion parece dar mejores resultados mse que friedman ya que estos son las peores configuracions segun los resultados que vemos, y por ultimo la tasa de aprendizaje parecen coincidir en 0.05 para las mejores configuraciones.

### 5.2.7 Histogram Gradient Boosting

In [ ]:
estimator = clone(hist_gradient_boosting_model)

In [ ]:
pipeline_Diabetes=make_pipeline(preproceserDiabetes,estimator)

In [ ]:
max_iter= [10,50,100]
learning_rate = [0.01, 0.05, 0.1]
min_samples_leaf=[10,15,20]


hist_gradient_boosting_clf_Diabetes = utils.optimize_params(pipeline_Diabetes,
                                                   X_train_Diabetes, y_train_Diabetes, cv,
                                                          histgradientboostingclassifier__max_iter= max_iter,
                                                   histgradientboostingclassifier__learning_rate=learning_rate,
                                                   histgradientboostingclassifier__min_samples_leaf=min_samples_leaf,scoring=['recall'],refit='recall')

En este caso parece ser que histograma da peores resultados de Gradiant Boosting, respecto a los hiperparametros la tasa de aprendizaje da mejores resultados utilizando la mayor, en cuanto al maximo numero de iteraciones  coincide con Wisconsin que da mejores resultados una mayor que una muy pequeña.

# 6. Construcción y validación del modelo final

Una vez obtenido los mejores estimadores para los distintos modelos que se consideran para el problema vamos a evaluar y considerar cual de ellos es el mejor para las bases de estudios que se están analizando utilizando los datos de test ya que hemos utilizado los datos de entrenamiento para generar dichos modelos:

## 6.1 Pima Indian Diabetes

In [ ]:
estimators_Diabetes = {
    "Nearest neighbors": k_neighbors_clf_Diabetes,
    "Decision tree": decision_tree_clf_Diabetes,
    "AdaBoost": adaboost_clf_Diabetes,
    "Bagging": bagging_clf_Diabetes,
    "Random Forests": random_forest_clf_Diabetes,
    "Gradient Boosting": gradient_boosting_clf_Diabetes,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf_Diabetes
}

In [ ]:
X_Diabetes = X_test_Diabetes
y_Diabetes = y_test_Diabetes

In [ ]:
utils.evaluate_estimators(estimators_Diabetes, X_Diabetes, y_Diabetes)

Viendo este análisis podemos concluir que segun el recall los modelos que mejor resultado dan los Arboles decision y Adaboost. Siendo estos el mismo resultado. Y el peor resultado que obtenemos es KNN.

## 6.2 Breast Cancer Wisconsin

In [ ]:
estimators_Wisconsin = {
    "Nearest neighbors": k_neighbors_clf_Wisconsin,
    "Decision tree": decision_tree_clf_Wisconsin,
    "AdaBoost": adaboost_clf_Wisconsin,
    "Bagging": bagging_clf_Wisconsin,
    "Random Forests": random_forest_clf_Wisconsin,
    "Gradient Boosting": gradient_boosting_clf_Wisconsin,
    "Histogram Gradient Boosting": hist_gradient_boosting_clf_Wisconsin
}

In [ ]:
X_Wisconsin = X_Wisconsin_test
y_Wisconsin = y_Wisconsin_test

In [ ]:
utils.evaluate_estimators(estimators_Wisconsin, X_Wisconsin, y_Wisconsin)

Viendo este análisis podemos concluir que segun el recall los modelos que mejor resultado dan los arboles decision y el siguiente Histogram Gradient Boosting y Gradient Boosting. Hay que tener en cuenta que en los primeros ejemplos hemos utilizado discretizacion aumentando asi nuestro resultados y tambien confirmamos que Histogram Gradient Boosting dan mejores resultados que Gradient Boosting. Y siendo Bagging el peor resultado obtenido.